### Load Temporally Streaming Tensor

In [270]:
import time
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac

In [429]:
def construct_tensor(factors):
    weights = tl.ones(factors[0].shape[1])
    est_tensor = tl.kruskal_to_tensor((weights, factors))
    return est_tensor
    
def print_tensor(X):
    print(np.round(X, 1))
    
def compare_tensors(A, B):
    print('||A-B||:', tl.norm(A - B))
    
def get_KhatriRao(factors):
    n_dim = len(factors)
    lefts = [factors[n_dim-1]]
    rights = [factors[0]]
    if n_dim > 2:
        for mode in range(1, n_dim-1):
            lefts.append(tl.tenalg.khatri_rao((lefts[mode-1], factors[n_dim-mode-1])))
            rights.append(tl.tenalg.khatri_rao((factors[mode], rights[mode-1])))
            
    K = lefts.copy()
    K[0] = lefts[n_dim-2]
    K.append(rights[n_dim-2].copy())
    if n_dim > 2:
        for mode in range(1, n_dim-1):
            K[mode] = tl.tenalg.khatri_rao((lefts[n_dim-mode-2], rights[mode-1]))
    return K
    
def get_Hadamard(factors):
    rank = factors[0].shape[1]
    H = tl.tensor(np.ones((rank, rank)))
    for factor in factors:
            H = H * tl.dot(tl.transpose(factor), factor)
    return H

### CP-ALS in 3rd dimension

In [90]:
from tensorly.decomposition.candecomp_parafac import initialize_factors, unfolding_dot_khatri_rao, KruskalTensor

def cp_als_3rd(X, rank, n_iter=100, verbose=False):
        
    factors = initialize_factors(X, rank, init='svd', svd='numpy_svd',
                                 random_state=None,
                                 non_negative=False,
                                 normalize_factors=False)
    weights = tl.ones(rank)

    A = factors[0]
    B = factors[1]
    C = factors[2]
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 0)
        A = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(A), A)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(B), B)*tl.dot(tl.transpose(A), A)
        mttkrp = unfolding_dot_khatri_rao(X, (None, [A, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, [A, B, C]))

def cp_als(X, rank, n_iter=100, verbose=False):
    factors = initialize_factors(X, rank, init='svd', svd='numpy_svd',
                                 random_state=None,
                                 non_negative=False,
                                 normalize_factors=False)
    weights = tl.ones(rank)
    n_dim = tl.ndim(X)
    
    for i in range(n_iter):
        for mode in range(n_dim):
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(factors):
                if j != mode:
                    V = V * tl.dot(tl.transpose(factor), factor)
                    
            mttkrp = unfolding_dot_khatri_rao(X, (None, factors), mode)
            factors[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        if verbose:
            X_est = construct_tensor(factors)
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, factors))

### OnlineCP in 3rd dimension 
*transformed from original updating A0*

In [491]:
def online_cp_als_3rd(factors_old, X_old, X_new, rank, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    A_old, B_old, C_old = factors_old
    X = np.concatenate((X_old, X_new))
    
    A0 = A_old
    B = B_old
    C = C_old
    
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, [A0, B, C]), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        V = tl.dot(tl.transpose(C), C)*(tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 1)
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        V = tl.dot(tl.transpose(B), B)*(tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 2)
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_old, (None, [A0, B, C]), 0)
        A0 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        A = np.concatenate((A0, A1))

        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)

    return KruskalTensor((weights, [A, B, C]))

def online_cp_als(factors_old, X_old, X_new, rank, n_iter=100, verbose=False, transformed=True):
    weights = tl.ones(rank)
    X = tl.tensor(np.concatenate((X_old, X_new)))
    n_dim = tl.ndim(X)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                if j != mode:
                    if j == 0:
                        V = V * (tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, U), mode)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

            
        if transformed:
            # temporal mode for A0
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                if j != 0:
                    V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp = unfolding_dot_khatri_rao(X_old, (None, U), 0)
            U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))


#TODO: optimize with complementary matrices P, Q, K, H
def online_cp_als_opt(factors_old, X_old, X_new, rank, n_iter=100, verbose=False, transformed=True):
    weights = tl.ones(rank)
    X = tl.tensor(np.concatenate((X_old, X_new)))
    n_dim = tl.ndim(X)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                if j != mode:
                    if j == 0:
                        V = V * (tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = unfolding_dot_khatri_rao(X_old, (None, U), mode)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))
        
        if transformed:
            # temporal mode for A0
            V = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                if j != 0:
                    V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp = unfolding_dot_khatri_rao(X_old, (None, U), 0)
            U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
            
        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))

import time
tensor = tl.tensor(np.arange(1200000, dtype='d').reshape((50, 40, 30, 20)))
X_old = tensor[:30,:,:,:]
X_new = tensor[30:,:,:,:]
rank = 3

X = np.concatenate((X_old, X_new))
# online_cp: 0.731866
start = time.time()
print('\n >> online_cp')
(weights, factors) = parafac(X_old, rank)
init_time = time.time()
print('init time:', init_time-start)
(weights, factors) = online_cp_als_opt(factors, X_old, X_new, rank, n_iter=10, verbose=True)
print('exec time:', time.time()-init_time)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])


 >> online_cp
init time: 0.5157718658447266
||A-B||: 305.1900795449966
||A-B||: 304.8814164852992
||A-B||: 304.5892797761524
||A-B||: 304.31372904255085
||A-B||: 304.05386166695865
||A-B||: 303.80882042391534
||A-B||: 303.57779159078757
||A-B||: 303.3600030635341
||A-B||: 303.1547225576151
||A-B||: 302.96125572099476
exec time: 0.30167436599731445
||A-B||: 302.96125572099476


### DTD in 3rd dimension

In [436]:
def dtd_als_3rd(factors_old, X_old, X_new, rank, mu=1, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    A_old, B_old, C_old = factors_old
    X = np.concatenate((X_old, X_new))
        
    A0 = A_old
    B = B_old
    C = C_old
    
    for i in range(n_iter):
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, [A0, B, C]), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        V = tl.dot(tl.transpose(C), C)*(mu*tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = mu*tl.dot(B_old, tl.dot(tl.transpose(C_old), C)*(tl.dot(tl.transpose(A_old), A0)))
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 1)
        B = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        V = tl.dot(tl.transpose(B), B)*(mu*tl.dot(tl.transpose(A0), A0) + tl.dot(tl.transpose(A1), A1))
        mttkrp0 = mu*tl.dot(C_old, tl.dot(tl.transpose(B_old), B)*(tl.dot(tl.transpose(A_old), A0)))
        mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, [A1, B, C]), 2)
        C = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))
        
        V = tl.dot(tl.transpose(C), C)*tl.dot(tl.transpose(B), B)
        mttkrp = tl.dot(A_old, tl.dot(tl.transpose(C_old), C)*tl.dot(tl.transpose(B_old), B))
        A0 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))
        
        A = np.concatenate((A0, A1))

        if verbose:
            X_est = construct_tensor([A, B, C])
            compare_tensors(X, X_est)
    
    return KruskalTensor((weights, [A, B, C]))

def dtd_als(factors_old, X_old, X_new, rank, mu=1, n_iter=100, verbose=False):
    weights = tl.ones(rank)
    X = np.concatenate((X_old, X_new))
    n_dim = tl.ndim(X)
    U = factors_old.copy()
    
    for i in range(n_iter):
        # temporal mode for A1
        V = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
        mttkrp = unfolding_dot_khatri_rao(X_new, (None, U), 0)
        A1 = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        # non-temporal mode
        for mode in range(1, n_dim):
            U1 = U.copy()
            U1[0] = A1
            V = tl.tensor(np.ones((rank, rank)))
            W = tl.tensor(np.ones((rank, rank)))
            for j, factor in enumerate(U):
                factor_old = factors_old[j]
                if j != mode:
                    W = W * tl.dot(tl.transpose(factor_old), factor)
                    if j == 0:
                        V = V * (mu*tl.dot(tl.transpose(factor), factor) + tl.dot(tl.transpose(A1), A1))
                    else:
                        V = V * tl.dot(tl.transpose(factor), factor)
            mttkrp0 = mu * tl.dot(factors_old[mode], W)
            mttkrp1 = unfolding_dot_khatri_rao(X_new, (None, U1), mode)
            U[mode] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp0 + mttkrp1)))

        # temporal mode for A0
        V = tl.tensor(np.ones((rank, rank)))
        W = tl.tensor(np.ones((rank, rank)))
        for j, factor in enumerate(U):
            factor_old = factors_old[j]
            if j != 0:
                V = V * tl.dot(tl.transpose(factor), factor)
                W = W * tl.dot(tl.transpose(factor_old), factor)
        mttkrp = tl.dot(factors_old[0], W)
        U[0] = tl.transpose(tl.solve(tl.transpose(V), tl.transpose(mttkrp)))

        if verbose:
            U1 = U.copy()
            U1[0] = np.concatenate((U[0], A1))
            X_est = construct_tensor(U1)
            compare_tensors(X, X_est)

    U[0] = np.concatenate((U[0], A1))
    return KruskalTensor((weights, U))

#### Testing for an example tensor

In [442]:
tensor = tl.tensor(np.arange(1200000, dtype='d').reshape((50, 40, 30, 20)))
X_old = tensor[:30,:,:,:]
X_new = tensor[30:,:,:,:]
rank = 3
X = np.concatenate((X_old, X_new))

# cp-als: 0.400312
start = time.time()
print('\n >> cp_als')
(weights, factors) = parafac(X, rank, verbose=False)
print('exec time:', time.time()-start)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])

# online_cp: 0.731866
start = time.time()
print('\n >> online_cp')
(weights, factors) = parafac(X_old, rank)
init_time = time.time()
print('init time:', init_time-start)
(weights, factors) = online_cp_als(factors, X_old, X_new, rank, n_iter=10, verbose=True)
print('exec time:', time.time()-init_time)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])

# dtd: 0.723(mu=1) 0.722692(mu=0.8)
start = time.time()
print('\n >> dtd')
(weights, factors) = parafac(X_old, rank)
init_time = time.time()
print('init time:', init_time-start)
(weights, factors) = dtd_als(factors, X_old, X_new, rank, mu=1, n_iter=10, verbose=True)
print('exec time:', time.time()-init_time)
X_est = construct_tensor(factors)
compare_tensors(X, X_est)
# print_tensor(X_est[:, 0, 0])



 >> cp_als
exec time: 0.1108243465423584
||A-B||: 1543.643400060986

 >> online_cp
init time: 0.06055927276611328
||A-B||: 2607.270899290528
||A-B||: 2604.722134109393
||A-B||: 2602.1871227457013
||A-B||: 2599.6429998513254
||A-B||: 2597.084646415563
||A-B||: 2594.506937611194
||A-B||: 2591.9047335766627
||A-B||: 2589.272870995526
||A-B||: 2586.606153842136
||A-B||: 2583.899345218078
exec time: 0.26463937759399414
||A-B||: 2583.899345218078

 >> dtd
init time: 0.05935859680175781
||A-B||: 2608.761342487092
||A-B||: 2606.8954514856237
||A-B||: 2605.218971022819
||A-B||: 2603.5410781245287
||A-B||: 2601.860951982082
||A-B||: 2600.177757731598
||A-B||: 2598.490664667816
||A-B||: 2596.7988460421125
||A-B||: 2595.101478644092
||A-B||: 2593.39774234584
exec time: 0.14069032669067383
||A-B||: 2593.39774234584


#### Testing for tensor stream

In [371]:
def create_tensor_stream(X, start_to_stream, batch_sizes):
    total_batch_size = np.sum(batch_sizes)
    if X.shape[0] != start_to_stream + total_batch_size:
        raise ValueError('Total batch size should be the size of streaming part of the tensor.')
    
    X_stream = [X[:start_to_stream]]
    batch_start = start_to_stream
    for batch_size in batch_sizes:
        batch_end = batch_start + batch_size
        X_stream.append(X[batch_start:batch_end])
        batch_start = batch_end
    return np.asarray(X_stream)

In [493]:
def online_cp_stream(X_stream, rank, verbose=False, transformed=True):
    print(' >> online_cp_stream')
    X_old = X_stream[0]
    start = time.time()
    (weights, factors) = parafac(X_old, rank)
    X_est = construct_tensor(factors)
    compare_tensors(X_old, X_est)
    init_time = time.time()
    print('init time:', init_time-start)
    for X_new in X_stream[1:]:
        (weights, factors) = online_cp_als(factors, X_old, X_new, rank, n_iter=1, verbose=False, transformed=transformed)
        X = np.concatenate((X_old, X_new))
        if verbose:
            print('exec time:', time.time()-init_time)
            X_est = construct_tensor(factors)
            compare_tensors(X, X_est)
        X_old = X
    print('exec time:', time.time()-init_time)
    X_est = construct_tensor(factors)
    compare_tensors(X_old, X_est)
    
    weights = tl.ones(rank)
    return KruskalTensor((weights, factors))
    
def dtd_stream(X_stream, rank, verbose=False):
    print(' >> dtd_stream')
    X_old = X_stream[0]
    start = time.time()
    (weights, factors) = parafac(X_old, rank)
    X_est = construct_tensor(factors)
    compare_tensors(X_old, X_est)
    init_time = time.time()
    print('init time:', init_time-start)
    for X_new in X_stream[1:]:
        (weights, factors) = dtd_als(factors, X_old, X_new, rank, n_iter=1, verbose=False)
        X = np.concatenate((X_old, X_new))
        if verbose:
            print('exec time:', time.time()-init_time)
            X_est = construct_tensor(factors)
            compare_tensors(X, X_est)
        X_old = X
    print('exec time:', time.time()-init_time)
    X_est = construct_tensor(factors)
    compare_tensors(X_old, X_est)
    
    weights = tl.ones(rank)
    return KruskalTensor((weights, factors))

In [495]:

# X = tl.tensor(np.arange(120000000, dtype='d').reshape((5000, 40, 20, 30)))
# X_stream = create_tensor_stream(tensor, start_to_stream=3000, batch_sizes=np.full((100), 20, dtype=int))

X = tl.tensor(np.arange(12000000, dtype='d').reshape((500, 40, 20, 30)))
X_stream = create_tensor_stream(X, start_to_stream=300, batch_sizes=np.full((10), 20, dtype=int))

# X = tl.tensor(np.arange(12000, dtype='d').reshape((500, 4, 2, 3)))
# X_stream = create_tensor_stream(X, start_to_stream=300, batch_sizes=np.full((10), 20, dtype=int))
(weights, factors) = online_cp_stream(X_stream, rank=5, verbose=False, transformed=False)
(weights, factors) = online_cp_stream(X_stream, rank=5, verbose=False, transformed=True)
(weights, factors) = dtd_stream(X_stream, rank=5, verbose=False)

 >> online_cp_stream
||A-B||: 3.5204351268609035
init time: 1.807868242263794
exec time: 2.56528902053833
||A-B||: 5.206025580002399
 >> online_cp_stream
||A-B||: 579.9550236206666
init time: 1.6144144535064697
exec time: 6.424799203872681
||A-B||: 54.21313558385349
 >> dtd_stream
||A-B||: 19.509695221547705
init time: 2.1080880165100098
exec time: 1.62351655960083
||A-B||: 31.32052811039929


### Load Sample Video Dataset

In [497]:
import csv
X = tl.tensor(np.zeros([205, 240, 320, 3], dtype='d'))

for i in range(41):
    start = time.time()
    with open('../sample_video/data/video{}.tensor'.format(i)) as file:
        reader = csv.reader(file, delimiter='\t')    
        for row in reader:
            indices = [[index] for index in np.int64(np.asarray(row[:-1]))-1]
            X[tuple(indices)] = np.double(row[-1])
    print('>> sample_video{} loaded '.format(i), time.time() - start)

>> sample_video0 loaded  22.637675046920776
>> sample_video1 loaded  22.592301845550537
>> sample_video2 loaded  22.62985134124756
>> sample_video3 loaded  21.97261667251587
>> sample_video4 loaded  22.10617995262146
>> sample_video5 loaded  22.068007469177246
>> sample_video6 loaded  21.868417501449585
>> sample_video7 loaded  22.063437461853027
>> sample_video8 loaded  21.826642990112305
>> sample_video9 loaded  22.06373429298401
>> sample_video10 loaded  21.938687086105347
>> sample_video11 loaded  21.968381881713867
>> sample_video12 loaded  22.025935649871826
>> sample_video13 loaded  22.17425775527954
>> sample_video14 loaded  22.044087648391724
>> sample_video15 loaded  22.14283776283264
>> sample_video16 loaded  22.013474464416504
>> sample_video17 loaded  22.16628932952881
>> sample_video18 loaded  22.035998344421387
>> sample_video19 loaded  22.226906776428223
>> sample_video20 loaded  21.84312343597412
>> sample_video21 loaded  21.989970684051514
>> sample_video22 loaded  21

In [498]:
X_stream = create_tensor_stream(X, start_to_stream=10, batch_sizes=np.full((39), 5, dtype=int))
(weights, factors) = online_cp_stream(X_stream, rank=3, verbose=True, transformed=False)
(weights, factors) = online_cp_stream(X_stream, rank=3, verbose=True, transformed=True)
(weights, factors) = dtd_stream(X_stream, rank=3, verbose=True)

 >> online_cp_stream
||A-B||: 39132.2477356458
init time: 3.686260461807251
exec time: 0.06327271461486816
||A-B||: 49032.172125730955
exec time: 0.24210214614868164
||A-B||: 57975.11977427322
exec time: 0.4605884552001953
||A-B||: 66177.24682465232
exec time: 0.7709648609161377
||A-B||: 74614.85835574944
exec time: 1.1158695220947266
||A-B||: 82112.74470728407
exec time: 1.5308318138122559
||A-B||: 88954.85441724073
exec time: 2.028876304626465
||A-B||: 95206.55811328608
exec time: 2.513704538345337
||A-B||: 100920.6401319217
exec time: 3.038317918777466
||A-B||: 106185.1594604483
exec time: 3.6597728729248047
||A-B||: 111126.96930012472
exec time: 4.237470388412476
||A-B||: 115633.19998240435
exec time: 4.872897624969482
||A-B||: 119790.49338122117
exec time: 5.530378341674805
||A-B||: 123759.02455950344
exec time: 6.252019643783569
||A-B||: 127502.71275877787
exec time: 6.984288692474365
||A-B||: 130939.21818348706
exec time: 7.7896246910095215
||A-B||: 134322.37818242577
exec time:

In [428]:
tensor = tl.tensor(np.arange(120, dtype='d').reshape((5, 4, 3, 2)))
(weights, factors) = parafac(tensor, rank=2)
X_est = construct_tensor(factors)
get_KhatriRao(factors)

[array([[-1.79554725e-01, -1.94641395e+02],
        [-1.93504327e-01, -1.58904079e+02],
        [-2.07453929e-01, -1.23166762e+02],
        [-2.21403530e-01, -8.74294456e+01],
        [-1.83840207e-01, -1.80351960e+02],
        [-1.98122748e-01, -1.47238269e+02],
        [-2.12405289e-01, -1.14124577e+02],
        [-2.26687830e-01, -8.10108861e+01],
        [-1.88125690e-01, -1.66062525e+02],
        [-2.02741170e-01, -1.35572459e+02],
        [-2.17356650e-01, -1.05082393e+02],
        [-2.31972130e-01, -7.45923266e+01],
        [-1.81660202e-01, -1.87104588e+02],
        [-1.95773379e-01, -1.52751074e+02],
        [-2.09886555e-01, -1.18397560e+02],
        [-2.23999731e-01, -8.40440461e+01],
        [-1.85995937e-01, -1.73368461e+02],
        [-2.00445956e-01, -1.41536982e+02],
        [-2.14895976e-01, -1.09705502e+02],
        [-2.29345995e-01, -7.78740229e+01],
        [-1.90331671e-01, -1.59632335e+02],
        [-2.05118534e-01, -1.30322890e+02],
        [-2.19905397e-01, -1.010